In [27]:
### get a prediction label for a new column called pred using Naive Bayes

In [1]:
# Todd McCullough
# 2020
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import cpl_main as cpl 
import random
import re

In [2]:
results = pd.read_csv(f'datasets/soccer-nn-train-NEWVALUES.csv')
results.head(2)

,game,team,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,r,s
0,I1,Forge FC,3.40,2.48,4.43,2.65,3.43,4.34,3.49,2.34,4.22,2.71,3.68,4.25,3.71,1.72,2,1
1,I1,York9 FC,4.09,4.06,2.42,2.71,3.46,2.30,3.55,2.83,3.64,1.93,4.76,3.70,4.47,3.45,2,1


In [3]:
gnb_db = results.copy()

In [4]:
a = []
for game in gnb_db['game'].unique():
    match = gnb_db[gnb_db['game'] == game]
    match = match.reset_index()
    for j in range(match.shape[0]):
        if j == 0:
            home = match.loc[j,['game','team','r','s']].values
            away = match.loc[j+1,['team','r','s']].values
            event = [home[0],home[1],home[2],home[3],away[0],away[1],away[2]]
            a.append(event)
        else:
            pass

In [5]:
print(results.shape,gnb_db.shape)

(3886, 18) (3886, 18)


In [6]:
db = pd.DataFrame(a,columns=['game','home','hr','hs','away','ar','as'])
db['hr'] = db['hr'].apply( lambda x: 'W' if x == 3 else 'D' if x == 2 else 'L')
db['ar'] = db['ar'].apply( lambda x: 'W' if x == 3 else 'D' if x == 2 else 'L')
db['d'] = db['game'].apply( lambda x: re.sub('[A-Z]+','',x))
db['m'] = 1

In [7]:
db.head(5)

,game,home,hr,hs,away,ar,as,d,m
0,I1,Forge FC,D,1,York9 FC,D,1,1,1
1,I2,Pacific FC,W,1,HFX Wanderers FC,L,0,2,1
2,I3,Pacific FC,L,1,Valour FC,W,2,3,1
3,I4,HFX Wanderers FC,W,2,Forge FC,L,1,4,1
4,I5,Cavalry FC,W,2,York9 FC,L,1,5,1


In [13]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
import statistics

In [14]:
def get_nb_prediction(query,x,y,result):
    mnb = MultinomialNB()
    # Train the model using the training sets
    mnb.fit(x,y)
    # use below instead of predicted = model.predict([result]) because we want the probability
    mnb_pred = np.round(mnb.predict_proba([result])[:, 1],decimals=2)
    pred = round(mnb_pred[0],2)
    return pred

def get_match_prediction_result(query,x,y,array):
    prediction = get_nb_prediction(query,x,y,array)
    return prediction

In [15]:
def get_match_prediction(q1,q2,x1,y1,x2,y2):
    if (len(x1) == 0) or (len(x2) == 0):
        x = round(1/3,2)
        home_win, away_win,draw = x,x,x
        return home_win,away_win,draw
    home_win = get_match_prediction_result(q1,x1,y1,[1,2])
    draw = get_match_prediction_result(q1,x1,y1,[1,1])
    away_win = get_match_prediction_result(q2,x2,y2,[2,2])
    return home_win, draw, away_win

In [16]:
q1,q2 = 'Toronto FC','Columbus'
t1_x,t1_y = cpl.get_NB_data(db,q1)
t2_x,t2_y = cpl.get_NB_data(db,q2)
home_win, draw, away_win = get_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y)
print(f'{q1} home win probability: {round(home_win,3)} \n{q2} away win probability: {round(away_win,3)} \nDraw probability: {round(draw,3)}')

Toronto FC home win probability: 0.43 
Columbus away win probability: 0.38 
Draw probability: 0.39


In [17]:
if home_win > away_win:
    hp, ap = 3, 1
elif home_win < away_win:
    hp, ap = 1, 3
else:
    hp, ap = 2, 2

In [18]:
hp, ap

(3, 1)

In [19]:
def get_team_comparison(data,q1,q2):
    # getting games with q1 in both home or away
    db = data[(data['home'] == q1) | (data['away'] == q1)]
    db = db.reset_index()
    db.pop('index')
    # filering down more to get only the games against q2
    db = db.sort_values(by=['m','d'],ascending=False)
    db = db[(db['home'] == q2) | (db['away'] == q2)]
    #print(db)
    db = db.reset_index()
    db.pop('index')
    if db.empty == True:
        db = pd.DataFrame([(0,0,0,0,q1,'D',q2,'D','empty',q1)],columns=['d','m','hs','as','home','hr','away','ar','summary','team'])
    if db.shape[0] > 15:
        return db.head(15)
    else:
        return db

In [21]:
c=[]
for i in range(db.shape[0]):
    q1 = db.loc[i]['home']
    q2 = db.loc[i]['away']
    t1_x,t1_y = cpl.get_NB_data(db.head(i),q1)
    t2_x,t2_y = cpl.get_NB_data(db.head(i),q2)
    home_win, draw, away_win = get_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y)
    game = db.loc[i]['game']
    print(f'game {game}\n{q1} home win probability: {round(home_win,3)} \n{q2} away win probability: {round(away_win,3)} \nDraw probability: {round(draw,3)}')
    if home_win > away_win:
        if home_win > draw:
            hp, ap = 3, 1
        else:
            hp, ap = 2, 2
    elif away_win > home_win:
        if away_win > draw:
            hp, ap = 3, 1
        else:
            hp, ap = 2, 2
    else:
        hp, ap = 2, 2
    print(hp,ap)
    c.append(hp)
    c.append(ap)

game I1
Forge FC home win probability: 0.33 
York9 FC away win probability: 0.33 
Draw probability: 0.33
2 2
game I2
Pacific FC home win probability: 0.33 
HFX Wanderers FC away win probability: 0.33 
Draw probability: 0.33
2 2
game I3
Pacific FC home win probability: 0.33 
Valour FC away win probability: 0.33 
Draw probability: 0.33
2 2
game I4
HFX Wanderers FC home win probability: 0.18 
Forge FC away win probability: 0.33 
Draw probability: 0.28
3 1
game I5
Cavalry FC home win probability: 0.33 
York9 FC away win probability: 0.33 
Draw probability: 0.33
2 2
game I6
Valour FC home win probability: 0.33 
FC Edmonton away win probability: 0.33 
Draw probability: 0.33
2 2
game I7
Forge FC home win probability: 0.24 
Pacific FC away win probability: 0.33 
Draw probability: 0.31
3 1
game I8
Cavalry FC home win probability: 0.43 
Valour FC away win probability: 0.34 
Draw probability: 0.33
3 1
game I9
Valour FC home win probability: 0.27 
HFX Wanderers FC away win probability: 0.34 
Draw 

game I76
York9 FC home win probability: 0.39 
Valour FC away win probability: 0.31 
Draw probability: 0.4
2 2
game I77
HFX Wanderers FC home win probability: 0.31 
Forge FC away win probability: 0.42 
Draw probability: 0.35
3 1
game I78
HFX Wanderers FC home win probability: 0.31 
Valour FC away win probability: 0.31 
Draw probability: 0.35
2 2
game I79
FC Edmonton home win probability: 0.39 
York9 FC away win probability: 0.39 
Draw probability: 0.38
2 2
game I80
Cavalry FC home win probability: 0.52 
Pacific FC away win probability: 0.37 
Draw probability: 0.4
3 1
game I81
HFX Wanderers FC home win probability: 0.31 
FC Edmonton away win probability: 0.38 
Draw probability: 0.35
3 1
game I82
Forge FC home win probability: 0.5 
Pacific FC away win probability: 0.37 
Draw probability: 0.41
3 1
game I83
York9 FC home win probability: 0.39 
Cavalry FC away win probability: 0.44 
Draw probability: 0.39
3 1
game I84
Cavalry FC home win probability: 0.54 
Valour FC away win probability: 0.3

game XIX56
Toronto FC home win probability: 0.52 
Chicago away win probability: 0.32 
Draw probability: 0.36
3 1
game XIX57
NY Red Bulls home win probability: 0.4 
Minnesota away win probability: 0.34 
Draw probability: 0.41
2 2
game XIX58
Orlando City home win probability: 0.38 
Colorado away win probability: 0.24 
Draw probability: 0.4
2 2
game XIX59
Columbus home win probability: 0.42 
New England away win probability: 0.41 
Draw probability: 0.35
3 1
game XIX60
Philadelphia home win probability: 0.44 
FC Dallas away win probability: 0.36 
Draw probability: 0.41
3 1
game XIX61
San Jose home win probability: 0.06 
Portland away win probability: 0.32 
Draw probability: 0.2
3 1
game XIX62
Seattle home win probability: 0.68 
Real Salt Lake away win probability: 0.37 
Draw probability: 0.52
3 1
game XIX63
FC Cincinnati home win probability: 0.48 
Sporting KC away win probability: 0.46 
Draw probability: 0.45
3 1
game XIX64
D.C. United home win probability: 0.51 
Montreal away win probabi

game XIX133
Seattle home win probability: 0.52 
Houston away win probability: 0.36 
Draw probability: 0.43
3 1
game XIX134
Atlanta home win probability: 0.36 
Orlando City away win probability: 0.4 
Draw probability: 0.34
3 1
game XIX135
D.C. United home win probability: 0.44 
Sporting KC away win probability: 0.41 
Draw probability: 0.38
3 1
game XIX136
Toronto FC home win probability: 0.36 
D.C. United away win probability: 0.4 
Draw probability: 0.34
3 1
game XIX137
Houston home win probability: 0.42 
Portland away win probability: 0.33 
Draw probability: 0.35
3 1
game XIX138
Vancouver home win probability: 0.34 
Atlanta away win probability: 0.35 
Draw probability: 0.36
2 2
game XIX139
Seattle home win probability: 0.52 
Orlando City away win probability: 0.39 
Draw probability: 0.43
3 1
game XIX140
Los Angeles FC home win probability: 0.57 
FC Dallas away win probability: 0.34 
Draw probability: 0.43
3 1
game XIX141
Montreal home win probability: 0.45 
New England away win probabi

game XIX206
Columbus home win probability: 0.26 
Orlando City away win probability: 0.36 
Draw probability: 0.31
3 1
game XIX207
D.C. United home win probability: 0.47 
Toronto FC away win probability: 0.35 
Draw probability: 0.41
3 1
game XIX208
Seattle home win probability: 0.47 
Vancouver away win probability: 0.41 
Draw probability: 0.42
3 1
game XIX209
San Jose home win probability: 0.42 
LA Galaxy away win probability: 0.38 
Draw probability: 0.4
3 1
game XIX210
Real Salt Lake home win probability: 0.37 
Sporting KC away win probability: 0.43 
Draw probability: 0.37
3 1
game XIX211
Portland home win probability: 0.36 
FC Dallas away win probability: 0.36 
Draw probability: 0.37
2 2
game XIX212
NYCFC home win probability: 0.56 
Seattle away win probability: 0.43 
Draw probability: 0.46
3 1
game XIX213
Orlando City home win probability: 0.36 
Philadelphia away win probability: 0.42 
Draw probability: 0.37
3 1
game XIX214
Chicago home win probability: 0.39 
Atlanta away win probabil

...

game XVI325
D.C. United home win probability: 0.42 
NYCFC away win probability: 0.41 
Draw probability: 0.4
3 1
game XVI326
Portland home win probability: 0.42 
Colorado away win probability: 0.37 
Draw probability: 0.39
3 1
game XVI327
San Jose home win probability: 0.34 
Vancouver away win probability: 0.36 
Draw probability: 0.36
2 2
game XVI328
Real Salt Lake home win probability: 0.38 
Sporting KC away win probability: 0.4 
Draw probability: 0.37
3 1
game XVI329
FC Dallas home win probability: 0.43 
Seattle away win probability: 0.4 
Draw probability: 0.39
3 1
game XVI330
Houston home win probability: 0.37 
LA Galaxy away win probability: 0.39 
Draw probability: 0.37
3 1
game XVI331
Seattle home win probability: 0.43 
Real Salt Lake away win probability: 0.37 
Draw probability: 0.39
3 1
game XVI332
Vancouver home win probability: 0.36 
Portland away win probability: 0.4 
Draw probability: 0.36
3 1
game XVI333
LA Galaxy home win probability: 0.4 
FC Dallas away win probability: 0.4

game XV58
Columbus home win probability: 0.39 
Orlando City away win probability: 0.36 
Draw probability: 0.38
3 1
game XV59
FC Dallas home win probability: 0.43 
Toronto FC away win probability: 0.41 
Draw probability: 0.39
3 1
game XV60
Colorado home win probability: 0.37 
Seattle away win probability: 0.4 
Draw probability: 0.38
3 1
game XV61
Real Salt Lake home win probability: 0.39 
Vancouver away win probability: 0.36 
Draw probability: 0.37
3 1
game XV62
LA Galaxy home win probability: 0.4 
Sporting KC away win probability: 0.4 
Draw probability: 0.38
2 2
game XV63
Philadelphia home win probability: 0.4 
New England away win probability: 0.39 
Draw probability: 0.38
3 1
game XV64
NYCFC home win probability: 0.45 
Portland away win probability: 0.39 
Draw probability: 0.39
3 1
game XV65
Chicago home win probability: 0.36 
NYCFC away win probability: 0.41 
Draw probability: 0.37
3 1
game XV66
Colorado home win probability: 0.37 
FC Dallas away win probability: 0.4 
Draw probabilit

game XV131
Philadelphia home win probability: 0.39 
Columbus away win probability: 0.38 
Draw probability: 0.38
3 1
game XV132
Montreal home win probability: 0.36 
Vancouver away win probability: 0.37 
Draw probability: 0.37
2 2
game XV133
Houston home win probability: 0.37 
NY Red Bulls away win probability: 0.41 
Draw probability: 0.38
3 1
game XV134
D.C. United home win probability: 0.42 
Toronto FC away win probability: 0.41 
Draw probability: 0.39
3 1
game XV135
Philadelphia home win probability: 0.39 
NYCFC away win probability: 0.4 
Draw probability: 0.38
3 1
game XV136
Columbus home win probability: 0.39 
Montreal away win probability: 0.37 
Draw probability: 0.38
3 1
game XV137
Sporting KC home win probability: 0.43 
Seattle away win probability: 0.4 
Draw probability: 0.39
3 1
game XV138
Chicago home win probability: 0.36 
Orlando City away win probability: 0.36 
Draw probability: 0.37
2 2
game XV139
LA Galaxy home win probability: 0.41 
Vancouver away win probability: 0.36 


game XV203
Columbus home win probability: 0.39 
Toronto FC away win probability: 0.41 
Draw probability: 0.38
3 1
game XV204
Montreal home win probability: 0.37 
Seattle away win probability: 0.4 
Draw probability: 0.37
3 1
game XV205
Chicago home win probability: 0.36 
New England away win probability: 0.39 
Draw probability: 0.37
3 1
game XV206
Houston home win probability: 0.37 
LA Galaxy away win probability: 0.39 
Draw probability: 0.38
3 1
game XV207
FC Dallas home win probability: 0.44 
Portland away win probability: 0.39 
Draw probability: 0.39
3 1
game XV208
NYCFC home win probability: 0.43 
Orlando City away win probability: 0.36 
Draw probability: 0.38
3 1
game XV209
D.C. United home win probability: 0.41 
Philadelphia away win probability: 0.38 
Draw probability: 0.39
3 1
game XV210
Vancouver home win probability: 0.37 
San Jose away win probability: 0.36 
Draw probability: 0.37
2 2
game XV211
NYCFC home win probability: 0.43 
Montreal away win probability: 0.37 
Draw proba

game XV275
Colorado home win probability: 0.37 
D.C. United away win probability: 0.4 
Draw probability: 0.38
3 1
game XV276
LA Galaxy home win probability: 0.41 
Montreal away win probability: 0.37 
Draw probability: 0.39
3 1
game XV277
San Jose home win probability: 0.35 
Seattle away win probability: 0.39 
Draw probability: 0.36
3 1
game XV278
Toronto FC home win probability: 0.44 
New England away win probability: 0.39 
Draw probability: 0.4
3 1
game XV279
Orlando City home win probability: 0.34 
Sporting KC away win probability: 0.4 
Draw probability: 0.37
3 1
game XV280
New England home win probability: 0.4 
NY Red Bulls away win probability: 0.41 
Draw probability: 0.39
3 1
game XV281
NYCFC home win probability: 0.42 
Toronto FC away win probability: 0.41 
Draw probability: 0.38
3 1
game XV282
San Jose home win probability: 0.35 
Montreal away win probability: 0.37 
Draw probability: 0.36
3 1
game XV283
Sporting KC home win probability: 0.42 
FC Dallas away win probability: 0.4 

In [22]:
print(len(c),results.shape)

3886 (3886, 18)


In [23]:
results['pred'] = c

In [24]:
s,t = [],[]
for i in range(results.shape[0]):
    if results.loc[i]['pred'] == results.loc[i]['r']:
        t.append(1)
    else:
        s.append(1)
print(round(len(s)/results.shape[0],2),round(len(t)/results.shape[0],2))

0.52 0.48


In [25]:
results.to_csv('datasets/soccer-nn-train-pred.csv',index=False)

In [26]:
results.head(50)

,game,team,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,r,s,pred
0,I1,Forge FC,3.40,2.48,4.43,2.65,3.43,4.34,3.49,2.34,4.22,2.71,3.68,4.25,3.71,1.72,2,1,2
1,I1,York9 FC,4.09,4.06,2.42,2.71,3.46,2.30,3.55,2.83,3.64,1.93,4.76,3.70,4.47,3.45,2,1,2
2,I2,Pacific FC,3.49,2.60,2.36,4.09,4.27,2.69,2.51,4.18,1.55,3.42,3.50,2.24,4.85,3.52,3,1,2
3,I2,HFX Wanderers FC,3.38,3.48,2.84,2.41,3.36,1.37,3.32,3.42,3.12,4.67,2.67,2.63,4.28,2.19,1,0,2
4,I3,Pacific FC,3.17,3.49,2.60,2.39,4.27,2.69,4.18,1.55,3.42,3.50,2.24,4.85,3.52,3.44,1,1,2
5,I3,Valour FC,2.42,3.41,3.34,3.33,2.61,3.10,3.55,2.62,3.40,3.17,2.63,4.06,4.26,1.55,3,2,2
6,I4,HFX Wanderers FC,3.38,3.48,2.84,2.41,3.36,1.37,3.32,3.42,3.12,4.26,4.67,2.67,2.63,3.11,3,2,3
7,I4,Forge FC,3.40,2.48,4.43,2.71,2.65,3.43,4.34,3.49,4.22,2.71,3.68,4.25,3.71,1.72,1,1,1
8,I5,Cavalry FC,2.26,2.83,4.70,3.46,3.45,2.38,4.39,3.53,3.68,1.79,3.19,3.36,4.43,3.38,3,2,2
9,I5,York9 FC,4.09,4.06,2.42,2.71,3.17,3.21,2.30,2.83,1.93,4.76,3.70,4.47,2.18,3.45,1,1,2
